## Сразу пишу код для выполнения первого и третьего задания

In [14]:
from bs4 import BeautifulSoup as bs
import pandas
from pprint import pprint
import requests as rq
import pandas as pd
import re
from pymongo import MongoClient
import json

In [15]:
client = MongoClient('127.0.0.1', 27017)
db = client['Vacancy']
sj = db.sj


## Парсинг с сайта SJ

In [16]:
def sal(sl):
    sl1 = sl.replace('\xa0','')
    sl2 = sl.split('\xa0')
    result = [int(i) for i in re.findall(r'\d+', sl1)]
    sal = [None, None, None]
    if len(result) == 2:
        sal[0],sal[1], sal[2] = result[0], result[1], sl2[-1]
    elif sl2[0] == 'от':
        sal[0], sal[2] = result[0], sl2[-1]
    elif sl2[0] == 'до' or sl2[0].isdigit():
        sal[1], sal[2] = result[0], sl2[-1]
      
    return sal

In [17]:

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}
vacancy = "Мерчендайзер"
location = "4"
sj_link = "https://www.superjob.ru/vacancy/search/"
page = 0
while True:
    page += 1
    sj_params = {"keywords":vacancy,
                 "geo%5Bt%5D%5B0%5D":location,
                 "page":page}
    sj_response = rq.get(sj_link,headers=header,params=sj_params)
    sj_soup = bs(sj_response.text,'lxml')
    vac = sj_soup.find_all('div',{'class':'Fo44F QiY08 LvoDO'})
    for serial in vac:
        web = 'sj'
        sj_id = "%s%s" % (web, serial.find('a',{'class':'icMQ_'})['href'][-13:-5])
        vac_name = serial.find('span',{'class':'_1rS-s'}).getText()
        salary = serial.find('span',{'class':'_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}).getText()
        sal_m = sal(salary)
        vac_link = sj_link[0:23] + serial.find('a')['href']
        
#         Данный блок оотвечает за добавление в БД только отсутствующих вакансийб на основании id сайта.
        data = {'_id': sj_id,
                'website': web, 
                'vacancy_name': vac_name, 
                'salary_full': salary.replace('\xa0',''), 
                'salary_from': sal_m[0], 
                'salary_to': sal_m[1],
                'curr': sal_m[2],
                'link': vac_link}
        db.sj.update_one({'_id': sj_id},  {"$set": data}, upsert=True)
#   --------------------------------------        
    if sj_soup.find('span', {'class':'qTHqo _1mEoj _2h9me DYJ1Y _2FQ5q _2GT-y'}).getText() == 'В начало':
        break

In [18]:
for i in sj.find({}):
    pprint(i)

{'_id': 'sj34030638',
 'curr': 'руб.',
 'link': 'https://www.superjob.ru/vakansii/merchendajzer-konsultant-34030638.html',
 'salary_from': 35000,
 'salary_full': '35000—42000руб.',
 'salary_to': 42000,
 'vacancy_name': 'Мерчендайзер',
 'website': 'sj'}
{'_id': 'sj33956541',
 'curr': 'руб.',
 'link': 'https://www.superjob.ru/vakansii/merchendajzer-33956541.html',
 'salary_from': 30000,
 'salary_full': '30000—33000руб.',
 'salary_to': 33000,
 'vacancy_name': 'Мерчендайзер',
 'website': 'sj'}
{'_id': 'sj32297210',
 'curr': 'руб.',
 'link': 'https://www.superjob.ru/vakansii/stacionarnyj-merchandajzer-32297210.html',
 'salary_from': 30000,
 'salary_full': 'от30000руб.',
 'salary_to': None,
 'vacancy_name': 'мерчандайзер',
 'website': 'sj'}
{'_id': 'sj32310600',
 'curr': 'руб.',
 'link': 'https://www.superjob.ru/vakansii/rabotnik-torgovogo-zala-32310600.html',
 'salary_from': 25000,
 'salary_full': 'от25000руб.',
 'salary_to': None,
 'vacancy_name': 'мерчендайзер',
 'website': 'sj'}
{'_id': 

## Парсинг с сайта HH

In [19]:
hh_link = "https://hh.ru/search/vacancy"
area = 1
st = 'searchVacancy'
fromSearch = 'true'
hh_from = 'suggest_post'
page=-1

while True:
    page +=1
    hh_params = {"area":area,
            "st":st,
            "text":vacancy,
            "fromSearch":fromSearch,
            "from": hh_from,
            "page":page}
    hh_response = rq.get(hh_link,headers=header,params=hh_params)
    hh_soup = bs(hh_response.text,'lxml')
    vac = hh_soup.find_all('div',{'class':'vacancy-serp-item'})
    
    for serial in vac:
        web = 'hh'
        hh_id ="%s%s" %  (web, json.loads(serial.find('script',{'data-name':'HH/VacancyResponsePopup/VacancyResponsePopup'})['data-params'])['vacancyId'])
        vac_name = serial.find('a',{'class':'bloko-link HH-LinkModifier'}).getText()
        salary = serial.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
        if salary is None:
            sal_m = [None, None, None]
        else:
            salary = serial.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).getText()
            sal_m = sal(salary.replace(' ','\xa0'))
        vac_link = serial.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        
#         Данный блок оотвечает за добавление в БД только отсутствующих вакансийб на основании id сайта.
        data = {'_id': hh_id, 
                'website': web, 
                'vacancy_name': vac_name, 
                'salary_full': salary, 
                'salary_from': sal_m[0], 
                'salary_to': sal_m[1],
                'curr': sal_m[2],
                'link': vac_link}
        db.hh.update_one({'_id': hh_id},  {"$set": data}, upsert=True)
#   --------------------------------------     
        
       
    if hh_soup.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}) is None:
        break


In [20]:
for i in db.hh.find({}):
    pprint(i)

{'_id': 'hh37708528',
 'curr': None,
 'link': 'https://hh.ru/vacancy/37708528?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': None,
 'salary_full': None,
 'salary_to': None,
 'vacancy_name': 'Мерчендайзер (м.Чертановская)',
 'website': 'hh'}
{'_id': 'hh37666835',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37666835?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 30000,
 'salary_full': '30\xa0000-40\xa0000 руб.',
 'salary_to': 40000,
 'vacancy_name': 'Мерчендайзер',
 'website': 'hh'}
{'_id': 'hh37528158',
 'curr': None,
 'link': 'https://hh.ru/vacancy/37528158?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': None,
 'salary_full': None,
 'salary_to': None,
 'vacancy_name': 'Продавец-кассир (ТРЦ Хорошо, ТЦ Авиапарк)',
 'website': 'hh'}
{'_id': 'hh37528204',
 'curr': None,
 'link': 'https://hh.ru/vacancy/37528204?query=%D0%9C%D0%B5%D1%80%D1%

 'link': 'https://hh.ru/vacancy/37332500?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 33500,
 'salary_full': 'от 33\xa0500 руб.',
 'salary_to': None,
 'vacancy_name': 'Мерчендайзер визитный (м.Минская, Новые Черёмушки)',
 'website': 'hh'}
{'_id': 'hh37727637',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37727637?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 32000,
 'salary_full': 'от 32\xa0000 руб.',
 'salary_to': None,
 'vacancy_name': 'Грузчик-мерчендайзер (м. Борисово)',
 'website': 'hh'}
{'_id': 'hh37513520',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37513520?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 50000,
 'salary_full': '50\xa0000-50\xa0000 руб.',
 'salary_to': 50000,
 'vacancy_name': 'Координатор отдела продаж',
 'website': 'hh'}
{'_id': 'hh35567298',
 'curr': None,
 'link': 'https://hh.ru/vacancy/35567298?quer

 'salary_full': '40\xa0000-45\xa0000 руб.',
 'salary_to': 45000,
 'vacancy_name': 'Работник торгового зала / грузчик в магазин (м. '
                 'Тимирязевская)',
 'website': 'hh'}
{'_id': 'hh37416751',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37416751?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 40000,
 'salary_full': '40\xa0000-45\xa0000 руб.',
 'salary_to': 45000,
 'vacancy_name': 'Работник торгового зала / грузчик в магазин (м. Ховрино)',
 'website': 'hh'}
{'_id': 'hh37385584',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37385584?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 35200,
 'salary_full': '35\xa0200-55\xa0400 руб.',
 'salary_to': 55400,
 'vacancy_name': 'Работник торгового зала',
 'website': 'hh'}
{'_id': 'hh36874068',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/36874068?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80'

In [21]:
def find_vac(sal_size, database):
    collections = database.list_collection_names()
    for col in collections:
        for i in db[col].find( {'$or':[{'salary_from': { '$gt': sal_size }} , {'salary_to': { '$gt': sal_size }}]}):
            pprint(i)

In [22]:
sal_size = 45000
database = db
find_vac(sal_size, database)

{'_id': 'hh37745520',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37745520?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 70000,
 'salary_full': 'от 70\xa0000 руб.',
 'salary_to': None,
 'vacancy_name': 'Региональный визуальный мерчендайзер',
 'website': 'hh'}
{'_id': 'hh37414056',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37414056?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 35000,
 'salary_full': '35\xa0000-46\xa0500 руб.',
 'salary_to': 46500,
 'vacancy_name': 'Мерчендайзер',
 'website': 'hh'}
{'_id': 'hh36582726',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/36582726?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 47000,
 'salary_full': '47\xa0000-47\xa0000 руб.',
 'salary_to': 47000,
 'vacancy_name': 'Мерчендайзер',
 'website': 'hh'}
{'_id': 'hh37635985',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37635985?quer

 'salary_from': 42500,
 'salary_full': '42\xa0500-50\xa0500 руб.',
 'salary_to': 50500,
 'vacancy_name': 'Мерчендайзер (р-н Речной вокзал-Алтуфьево)',
 'website': 'hh'}
{'_id': 'hh37689266',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37689266?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 80000,
 'salary_full': 'от 80\xa0000 руб.',
 'salary_to': None,
 'vacancy_name': 'Супервайзер (САО, СВАО)',
 'website': 'hh'}
{'_id': 'hh37693333',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37693333?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 56500,
 'salary_full': '56\xa0500-56\xa0500 руб.',
 'salary_to': 56500,
 'vacancy_name': 'Визуальный мерчендайзер',
 'website': 'hh'}
{'_id': 'hh37689230',
 'curr': 'руб.',
 'link': 'https://hh.ru/vacancy/37689230?query=%D0%9C%D0%B5%D1%80%D1%87%D0%B5%D0%BD%D0%B4%D0%B0%D0%B9%D0%B7%D0%B5%D1%80',
 'salary_from': 80000,
 'salary_full': 'от 80\xa0000 руб